# Analysing an Interest Rate Swap - Customising Schedules

You can override the schedule dates if you need to.

In [1]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 22:32 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [2]:
value_dt = Date(20, 10, 2015)

In [3]:
discount_curve = DiscountCurveFlat(value_dt, 
                                     0.01,
                                     FrequencyTypes.CONTINUOUS,
                                     DayCountTypes.ACT_365F)

In [4]:
index_curve = DiscountCurveFlat(value_dt, 
                                  0.02,
                                  FrequencyTypes.CONTINUOUS,
                                  DayCountTypes.ACT_365F)

## Creating the Swap

In [5]:
settle_dt = value_dt.add_weekdays(5)

In [6]:
settle_dt

27-OCT-2015

In [7]:
maturity_dt = settle_dt.add_tenor("10Y")

We begin by setting the market conventions we wish to use.

In [8]:
swap_cal_type = CalendarTypes.UNITED_STATES
bd_type = BusDayAdjustTypes.MODIFIED_FOLLOWING
dg_type = DateGenRuleTypes.FORWARD

We then define the Fixed Leg

In [9]:
fixed_cpn = 0.025
fixed_freq_type = FrequencyTypes.SEMI_ANNUAL
fixed_dc_type = DayCountTypes.ACT_360

And then we define the Floating Leg

In [10]:
float_spread = 0.004
float_freq_type = FrequencyTypes.QUARTERLY
float_dc_type = DayCountTypes.ACT_360

We need to specify the size and direction of the swap.

In [11]:
swap_type = SwapTypes.PAY
notional = 10 * ONE_MILLION

And then we define when the swap begins and ends.

We can now create the swap object.

In [12]:
swap = IborSwap(settle_dt,
                    maturity_dt,
                    swap_type,
                    fixed_cpn,
                    fixed_freq_type,
                    fixed_dc_type,
                    notional,
                    float_spread,
                    float_freq_type,
                    float_dc_type,
                    swap_cal_type,
                    bd_type,
                    dg_type)

In [13]:
swap.value(value_dt, discount_curve, index_curve)

-88571.26347494917

In [14]:
swap.print_fixed_leg_pv()

START DATE: 27-OCT-2015
MATURITY DATE: 27-OCT-2025
COUPON (%): 2.5
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
+---------+-------------+----------+------+-----------+--------+-----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL | RATE |    PMNT   |   DF   |     PV    |   CUM_PV   |
+---------+-------------+----------+------+-----------+--------+-----------+------------+
|    1    | 27-APR-2016 | 10000000 | 2.5  | 127083.33 | 0.9948 | 126423.52 | 126423.52  |
|    2    | 27-OCT-2016 | 10000000 | 2.5  | 127083.33 | 0.9898 | 125791.26 | 252214.78  |
|    3    | 27-APR-2017 | 10000000 | 2.5  | 126388.89 | 0.9849 | 124481.62 |  376696.4  |
|    4    | 27-OCT-2017 | 10000000 | 2.5  | 127083.33 |  0.98  | 124539.62 | 501236.02  |
|    5    | 27-APR-2018 | 10000000 | 2.5  | 126388.89 | 0.9751 | 123243.01 | 624479.03  |
|    6    | 29-OCT-2018 | 10000000 | 2.5  | 128472.22 | 0.9702 | 124641.14 | 749120.17  |
|    7    | 29-APR-2019 | 10000

In [15]:
swap.print_float_leg_pv()

START DATE: 27-OCT-2015
MATURITY DATE: 27-OCT-2025
SPREAD (BPS): 40.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
+---------+-------------+----------+--------+----------+--------+----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL |  IBOR  |   PMNT   |   DF   |    PV    |   CUM_PV   |
+---------+-------------+----------+--------+----------+--------+----------+------------+
|    1    | 27-JAN-2016 | 10000000 | 2.005  | 61462.38 | 0.9973 | 61295.9  |  61295.9   |
|    2    | 27-APR-2016 | 10000000 | 2.005  | 60792.92 | 0.9948 | 60477.28 | 121773.18  |
|    3    | 27-JUL-2016 | 10000000 | 2.005  | 60792.92 | 0.9923 | 60326.69 | 182099.88  |
|    4    | 27-OCT-2016 | 10000000 | 2.005  | 61462.38 | 0.9898 | 60837.48 | 242937.36  |
|    5    | 27-JAN-2017 | 10000000 | 2.005  | 61462.38 | 0.9873 | 60684.33 | 303621.69  |
|    6    | 27-APR-2017 | 10000000 | 2.0049 | 60123.49 | 0.9849 | 59216.2  | 362837.89  |
|    7    | 27-JUL-2017 | 1000

## Customising the Schedules

Suppose I want to customise the schedules in a way that is not possible using the IborSwap constructor. This can easily be done by overriding the internal variable that stores the schedule of dates. For example, suppose I wish to make the fixed leg have odd payments as follows.

In [16]:
fixedSchedule = [value_dt, Date(15, 11, 2015), Date(15,11, 2020), Date(15,11,2025)]

The first date is assumed to be the previous cpn date. We can set the schedule easily.

In [17]:
swap._adjusted_fixed_dates = fixedSchedule

Now I redo the valuation

In [18]:
swap.value(value_dt, discount_curve, index_curve)

-88571.26347494917

We can then check to see how the valuation is arrived at.

In [19]:
swap.print_fixed_leg_pv()

START DATE: 27-OCT-2015
MATURITY DATE: 27-OCT-2025
COUPON (%): 2.5
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
+---------+-------------+----------+------+-----------+--------+-----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL | RATE |    PMNT   |   DF   |     PV    |   CUM_PV   |
+---------+-------------+----------+------+-----------+--------+-----------+------------+
|    1    | 27-APR-2016 | 10000000 | 2.5  | 127083.33 | 0.9948 | 126423.52 | 126423.52  |
|    2    | 27-OCT-2016 | 10000000 | 2.5  | 127083.33 | 0.9898 | 125791.26 | 252214.78  |
|    3    | 27-APR-2017 | 10000000 | 2.5  | 126388.89 | 0.9849 | 124481.62 |  376696.4  |
|    4    | 27-OCT-2017 | 10000000 | 2.5  | 127083.33 |  0.98  | 124539.62 | 501236.02  |
|    5    | 27-APR-2018 | 10000000 | 2.5  | 126388.89 | 0.9751 | 123243.01 | 624479.03  |
|    6    | 29-OCT-2018 | 10000000 | 2.5  | 128472.22 | 0.9702 | 124641.14 | 749120.17  |
|    7    | 29-APR-2019 | 10000

Clearly the cpns have been adjusted. We do not the ability to override the flow amounts.

The floating leg is not affected.

Copyright (c) 2020 Dominic O'Kane